<a href="https://colab.research.google.com/github/parthib22/colaboratory/blob/main/File2Vid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow numpy

In [ ]:
from PIL import Image
import numpy as np
import os

**ENCODING**

In [ ]:
def get_ext(file_path):
    extension = ''
    ptr = len(file_path) - 1

    while ptr > -1 and file_path[ptr] != '.':
        ptr -= 1

    if ptr > -1:
        extension = file_path[ptr:]

    return extension

In [ ]:
def f2b(file_path):
    try:
        with open(file_path, 'rb') as file:
            binary_data = file.read()
            binary_string = ''.join(format(byte, '08b') for byte in binary_data)
            return binary_string
    except FileNotFoundError:
        return "File not found."
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
def b2i(_bin, width, height, temp_dir='temp'):
    # Total pixels per image
    pixels_per_image = width * height

    # Split the binary string into chunks of size pixels_per_image
    num_images = len(_bin) // pixels_per_image
    remainder = len(_bin) % pixels_per_image

    if not os.path.exists(temp_dir):
      os.makedirs(temp_dir)

    # Create images for each chunk
    for i in range(num_images):
        chunk = _bin[i * pixels_per_image:(i + 1) * pixels_per_image]
        create_image_from_binary(temp_dir, chunk, width, height, f"{i + 1}.png")

    # Handle the remainder (if any)
    if remainder > 0:
        chunk = _bin[num_images * pixels_per_image:]
        # Pad the remaining chunk to fit into a full image
        chunk = chunk.ljust(pixels_per_image, '0')  # Pad with '0's
        create_image_from_binary(temp_dir, chunk, width, height, f"{num_images + 1}.png")

    return f"Generated {num_images + (1 if remainder > 0 else 0)} image(s)."

def create_image_from_binary(temp_dir, binary_chunk, width, height, output_file):
    # Convert the binary string to a NumPy array of pixel values
    pixel_values = np.array([255 if bit == '1' else 0 for bit in binary_chunk], dtype=np.uint8)

    # Reshape the array to match the image dimensions
    image_array = pixel_values.reshape((height, width))

    # Create and save the image
    image = Image.fromarray(image_array)
    save_path = os.path.join(temp_dir, os.path.basename(output_file))
    image.save(save_path)
    image

In [ ]:
# ONLY TO DELETE THE TEMP FOLDER
import shutil

def del_temp():
    shutil.rmtree('temp')

In [ ]:
import cv2
import os
# from glob import glob

def i2v(folder_path, output_video_path, fps=30):
    # Get all image file paths in the folder
    # image_files = sorted(glob(os.path.join(folder_path, "*.png")))  # Change to "*.jpg" if needed

    files = len(os.listdir(folder_path))
    image_files = [os.path.join(folder_path, os.path.basename(f"{i + 1}.png")) for i in range(files)]

    if not image_files:
        print("No images found in the folder.")
        return

    # Read the first image to get the dimensions
    first_image = cv2.imread(image_files[0])
    height, width, _ = first_image.shape

    # Define the video codec and create VideoWriter object
    # Preserve the original bitrate (optional, depends on codec support)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' or other codecs if needed
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    print(f"Creating video: {output_video_path} with FPS: {fps}")

    for image_file in image_files:
        # Read each image
        frame = cv2.imread(image_file)

        # Ensure the dimensions match
        if frame.shape[:2] != (height, width):
            print(f"Skipping {image_file}: Dimensions do not match!")
            continue

        # Write the frame to the video
        out.write(frame)

    # Release the video writer
    out.release()
    print("Video created successfully!")


**DECODING**

In [ ]:
import cv2
import os

def v2i(video_path, output_folder="temp"):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    # Frame counter
    frame_count = 0

    print(f"Extracting frames from video: {video_path}")
    while True:
        # Read the next frame
        ret, frame = cap.read()
        if not ret:
            break  # Break if there are no more frames

        # Save the frame as an image
        frame_count += 1
        output_path = os.path.join(output_folder, f"{frame_count}.png")
        cv2.imwrite(output_path, frame)

        print(f"Saved frame {frame_count} as {output_path}")

    # Release the video capture
    cap.release()
    print(f"Extraction complete. {frame_count} frames saved in '{output_folder}'.")


In [ ]:
def i2b(image_paths):
    _bin = ""

    for image_path in image_paths:
        # Open the image and convert it to grayscale
        image = Image.open(image_path).convert('L')  # Convert to grayscale

        # Convert the image to a NumPy array
        image_array = np.array(image)

        # Map pixel values to binary:
        # - White (255) -> '1'
        # - Black (0) -> '0'
        binary_chunk = ''.join('1' if pixel == 255 else '0' for pixel in image_array.flatten())

        # Append the binary chunk to the full binary string
        _bin += binary_chunk

    return _bin


In [ ]:
def b2f(binary_string, output_file_path):
    try:
        # Split the binary string into chunks of 8 bits
        byte_data = [binary_string[i:i + 8] for i in range(0, len(binary_string), 8)]

        # Convert each chunk of 8 bits to a byte
        byte_array = bytearray(int(byte, 2) for byte in byte_data)

        # Write the byte array to a file
        with open(output_file_path, 'wb') as file:
            file.write(byte_array)

        return f"File has been written to {output_file_path}"
    except Exception as e:
        return f"An error occurred: {e}"

**MAIN**

In [ ]:
file_path = '1093662-hd_1920_1080_30fps.mp4'

extension = get_ext(file_path)
print(extension)

_bin = f2b(file_path)

print(len(_bin))

temp_dir = "temp"

result = b2i(_bin, 1280, 720, temp_dir)
print(result)

folder_path = "temp"
output_video_path = "encoded.mp4"
i2v(folder_path, output_video_path, fps=30)

del_temp()

.mp4
41564968
Generated 46 image(s).
Creating video: encoded.mp4 with FPS: 30
Video created successfully!


In [ ]:
video_path = "encoded.mp4"
v2i(video_path)

# image_paths = [f"image_{i+1}.png" for i in range(21)]  # Replace with your
files = len(os.listdir('temp'))
print(files)

temp_dir = 'temp'

image_paths = [os.path.join(temp_dir,os.path.basename(f"{i + 1}.png")) for i in range(files)]
print(image_paths)
# image file paths
_bin2 = i2b(image_paths)
print(f"Binary String Length: {len(_bin2)}")
print(_bin == _bin2[:len(_bin)])

output_file_path = 'hehehe' + extension
result = b2f(_bin2, output_file_path)

del_temp()

Extracting frames from video: encoded.mp4
Saved frame 1 as temp/1.png
Saved frame 2 as temp/2.png
Saved frame 3 as temp/3.png
Saved frame 4 as temp/4.png
Saved frame 5 as temp/5.png
Saved frame 6 as temp/6.png
Saved frame 7 as temp/7.png
Saved frame 8 as temp/8.png
Saved frame 9 as temp/9.png
Saved frame 10 as temp/10.png
Saved frame 11 as temp/11.png
Saved frame 12 as temp/12.png
Saved frame 13 as temp/13.png
Saved frame 14 as temp/14.png
Saved frame 15 as temp/15.png
Saved frame 16 as temp/16.png
Saved frame 17 as temp/17.png
Saved frame 18 as temp/18.png
Saved frame 19 as temp/19.png
Saved frame 20 as temp/20.png
Saved frame 21 as temp/21.png
Saved frame 22 as temp/22.png
Saved frame 23 as temp/23.png
Saved frame 24 as temp/24.png
Saved frame 25 as temp/25.png
Saved frame 26 as temp/26.png
Saved frame 27 as temp/27.png
Saved frame 28 as temp/28.png
Saved frame 29 as temp/29.png
Saved frame 30 as temp/30.png
Saved frame 31 as temp/31.png
Saved frame 32 as temp/32.png
Saved frame 33 a

it didnot work as planned, the vid is not playing